<a href="https://colab.research.google.com/github/dtype2100/TIL/blob/master/Top2Vec_DataFrame_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Top2Vec install

In [1]:
pip install top2vec

In [2]:
pip install top2vec[sentence_encoders]

# 데이터 불러오기

In [3]:
import pandas as pd

In [8]:
df1 = pd.read_excel('/content/drive/MyDrive/NLP/Keyword_extraction/naver_kakao/네이버_증권_산업기업_20170101-20210531.xlsx')
df2 = pd.read_excel('/content/drive/MyDrive/NLP/Keyword_extraction/naver_kakao/네이버_증권_산업기업_20210601-20220430.xlsx')

/usr/local/lib/python3.7/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


# 데이터 전처리

In [18]:
df = pd.concat([df1, df2]) # 파일 합치기 (종합할 데이터 없으면 각주처리)
title_df = df[df['제목'].str.contains('네이버')] # 해당 키워드를 포함한 제목 추출 (불러온 데이터에 따라 키워드 변경)
title_df.drop_duplicates(subset=['제목'], inplace=True) # 중복 제거
new_df = pd.DataFrame({'일자' : pd.to_datetime(title_df['일자'], format='%Y%m%d'), '제목' : title_df['제목'], 'URL' : title_df['URL']}) # datetime 데이터프레임 생성
mask = (new_df['일자'] >= '2021-04-01') & (new_df['일자'] <= '2021-06-30') # 날짜 변경
period_df = new_df.loc[mask] # 날짜 인덱싱
lis_df = list(period_df['제목']) # 데이터 타입 list로 변경

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


# Top2Vec

In [19]:
from top2vec import Top2Vec

# 모델 훈련

In [20]:
model = Top2Vec(documents=lis_df, speed="learn", workers=8, embedding_model='universal-sentence-encoder')

2022-05-10 02:41:34,081 - top2vec - INFO - Pre-processing documents for training
INFO:top2vec:Pre-processing documents for training
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
2022-05-10 02:41:34,113 - top2vec - INFO - Downloading universal-sentence-encoder model
INFO:top2vec:Downloading universal-sentence-encoder model
2022-05-10 02:41:37,404 - top2vec - INFO - Creating joint document/word embedding
INFO:top2vec:Creating joint document/word embedding
2022-05-10 02:41:37,771 - top2vec - INFO - Creating lower dimension embedding of documents
INFO:top2vec:Creating lower dimension embedding of documents
2022-05-10 02:41:41,221 - top2vec - INFO - Finding dense areas of documents
INFO:top2vec:Finding dense areas of documents
2022-05-10 02:41:41,241 - top2v

# 토픽 수 확인

In [21]:
model.get_num_topics()

2

# 토픽 확인, 검색 토픽 설정, Topic num 설정

In [26]:
topic_sizes, topic_nums = model.get_topic_sizes()
topic_words, word_scores, topic_scores, topic_nums = model.search_topics(keywords=["네이버"], num_topics=2) # num_topics는 'model.get_num_topics()'에 나온 수와 동일하게 설정. keywords에 확인할 키워드 설정.

In [27]:
topic_words # 토픽 확인

[array(['네이버', '이베이코리아', '신세계', '시총', '카카오'], dtype='<U6'),
 array(['네이버', '이베이코리아', '카카오', '신세계', '시총'], dtype='<U6')]

# 유사도 높은 뉴스 출력 및 Score 확인

In [29]:
documents, document_scores, document_ids = model.search_documents_by_topic(topic_num=1, num_docs=16) # 필요한 만큼 topic_num 변경, num_docs 변경
for doc, score, doc_id in zip(documents, document_scores, document_ids):
    print(f"Document: {doc_id}, Score: {score}")
    print("-----------")
    print(doc)
    print("-----------")
    print()

Document: 495, Score: 0.8380758762359619
-----------
`신세계 네이버 vs 롯데` 이베이 인수 2파전
-----------

Document: 96, Score: 0.8342318534851074
-----------
카카오 vs 네이버, 판커지는 ‘선물하기’ 시장[언박싱]
-----------

Document: 493, Score: 0.8323456048965454
-----------
빅테크 분석서 '네이버 vs 카카오' 인기
-----------

Document: 259, Score: 0.8315381407737732
-----------
요동치는 e커머스 시장 신세계 VS 쿠팡 VS 네이버
-----------

Document: 242, Score: 0.8147686719894409
-----------
이베이 인수 이마트, 네이버 쿠팡과 新3강 게임 체인저 vs 승자의 저주
-----------

Document: 391, Score: 0.8124184012413025
-----------
e커머스 업계 3강 구도 재편? 신세계 네이버 vs 쿠팡
-----------

Document: 73, Score: 0.8058498501777649
-----------
네이버 vs 카카오, 글로벌 콘텐츠 전쟁 서막 올랐다
-----------

Document: 457, Score: 0.805296778678894
-----------
카카오 vs 네이버 시총 3위 놓고 진검승부 펼친다
-----------

Document: 264, Score: 0.7952497005462646
-----------
[투자뉴스7] '국민주' 카카오 VS '큰 손 사랑' 네이버 "자식에게 물려줄거면 "
-----------

Document: 21, Score: 0.7858288884162903
-----------
‘네이버 vs 카카오’ 몸집전쟁 불 붙었다
-----------

Document: 494, Score: 0.781

# Top2Vec으로 추출한 문서를 데이터프레임으로 만들기

In [35]:
doc_df = pd.DataFrame({'일자' : document_ids, '제목' : documents, 'URL' : document_scores}) # Top2Vec을 데이터프레임으로 만들기
sum_df = pd.concat([doc_df, period_df]) # Top2Vec에서 추출한 데이터와 날짜 인덱싱한 데이터와 합치기
dupl_df = sum_df[sum_df['제목'].duplicated()] # Top2Vec에서 추출한 데이터와 날짜 인덱싱한 데이터에서 중복값 변수 설정
date_df = pd.DataFrame({'일자' : pd.to_datetime(dupl_df['일자'], format='%Y%m%d'), '제목' : dupl_df['제목'], 'URL' : dupl_df['URL']}) # 중복값 설정한 데이터에 datetime 순서로 데이터프레임 생성
sort_df = date_df.sort_values(by=['일자']) # 데이터프레임을 일자별로 정렬

In [36]:
sort_df.info() # 데이터 확인

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16 entries, 1076 to 5367
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일자      16 non-null     datetime64[ns]
 1   제목      16 non-null     object        
 2   URL     16 non-null     object        
dtypes: datetime64[ns](1), object(2)
memory usage: 512.0+ bytes


# 추출한 데이터 엑셀로 저장

In [ ]:
# sort_df.to_excel('네이버_증권_산업기업_20210401_20210631.xlsx') # 엑셀로 저장